## Few-shot settings
Given few shot data, we do not fine-tune the detector. Instead, we treat the model trained in the first satge as feature extractor, which is equalevent to few-shot learning.

In adapting to new LLMs, the model may get kshot data of the previous classes.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [3]:
import sys
sys.path.append('..')

load_incremental_topic can be used by formatting the order

---
```python
# two stages. first stage includes 4 classes (with human) and the second stage incude 2 classes
order = [['gpt35', 'Mixtral','Moonshot',], ['Llama3','gpt-4omini']]
```
---
```python
# fives stages. first stage includes 2 classes (with human) and each remaining stage includes 1 class
order = [['Moonshot'],['Mixtral'],['gpt35'],['Llama3'],['gpt-4omini']]
```

In [5]:
order = [['Moonshot'],['Mixtral'],['gpt35'],['Llama3'],['gpt-4omini']]
order = [['gpt35', 'Mixtral','Moonshot','Llama3'],['gpt-4omini']]
from mgtbench.loading import load_incremental_topic, load_incremental

data = load_incremental_topic(order, "Social_sciences")
# data = load_incremental(order, "Art")

In [6]:
set(data['train'][0]['label']), set(data['train'][1]['label'])

({0, 1, 2, 3, 4}, {5})

In [8]:
import torch
import numpy as np
import random
import os
def setup_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
setup_seed(3407)

In [7]:
nclass = len(set(data['train'][0]['label']))
nclass

5

RelationNet

In [6]:
from mgtbench import AutoDetector, AutoExperiment
method = 'rn'
model= '/data1/models/roberta-base'
nclass = len(set(data['train'][0]['label']))

metric = AutoDetector.from_detector_name(method, model_name_or_path=model,num_labels=nclass,kshot=5)
experiment = AutoExperiment.from_experiment_name('fewshot',detector=[metric])
config = {'need_finetune': True,
          'need_save': False,
          'epochs': 1,
          'lr': 5e-6,
          'batch_size':64,
          'save_path': '/data1/lyl/mgtout-1/',
          'eval':True,
          'lr_factor': 4,
          'kshot':20,
          'classifier':'MLP'
        }
# cfg = SupervisedConfig()
# cfg.update(config)
experiment.load_data(data)
res = experiment.launch(**config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /data1/models/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


length is set to 512
Calculate result for each data point
Running prediction of detector rn


/home/lyl/miniconda3/envs/llm/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/lyl/miniconda3/envs/llm/lib/python3.10/site-packages/accelerate/accelerator.py:457: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the ke

Step,Training Loss
50,0.902800


100%|██████████| 90/90 [00:00<00:00, 115.09it/s]


0.2603793740272522
0.10639829188585281
0.06096925958991051
0.02816082537174225
0.021494131535291672
0.0206310972571373
Fine-tune finished
Predict testing data


100%|██████████| 2308/2308 [00:20<00:00, 114.65it/s]


torch.Size([2308, 768]) torch.Size([768])
Run classification for results
[[357  33   0   0   0   0]
 [  4 195  33  19  29 110]
 [  0  21 245  48  73   3]
 [  0   4  45 223  97  21]
 [  1  31 108  73 117  28]
 [  0  37  26  55 101 171]]


In [7]:
res

[DetectOutput(name='test_pred', train=None, test=Metric(acc=0.5667244367417678, precision=0.580324360793989, recall=0.5667244367417678, f1=0.5709198209659895, auc=-1.0, conf_m=array([[357,  33,   0,   0,   0,   0],
        [  4, 195,  33,  19,  29, 110],
        [  0,  21, 245,  48,  73,   3],
        [  0,   4,  45, 223,  97,  21],
        [  1,  31, 108,  73, 117,  28],
        [  0,  37,  26,  55, 101, 171]])))]

distribution calibration

In [8]:
from mgtbench import AutoDetector, AutoExperiment
method = 'generate'
model= '/data1/models/roberta-base'
nclass = len(set(data['train'][0]['label']))

metric = AutoDetector.from_detector_name(method, model_name_or_path=model,num_labels=nclass,kshot=5)
experiment = AutoExperiment.from_experiment_name('fewshot',detector=[metric])
config = {'need_finetune': True,
          'need_save': False,
          'epochs': 2,
          'lr': 5e-6,
          'batch_size':64,
          'save_path': '/data1/lyl/mgtout-1/',
          'eval':True,
          'lr_factor': 4,
          'kshot':20,
          'classifier':'MLP'
        }
# cfg = SupervisedConfig()
# cfg.update(config)
experiment.load_data(data)
res = experiment.launch(**config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /data1/models/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


length is set to 512
Calculate result for each data point
Running prediction of detector generate


/home/lyl/miniconda3/envs/llm/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/lyl/miniconda3/envs/llm/lib/python3.10/site-packages/accelerate/accelerator.py:457: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the ke

Step,Training Loss
50,0.958000
100,0.591700
150,0.394300


training Macro F1 Score: 0.9893
Fine-tune finished
Predict testing data
Run classification for results
[[387   3   0   0   0   0]
 [ 11 340  10  15   2  12]
 [  5  20 277  79   8   1]
 [  1   0  28 348   2  11]
 [  2  26 151 169   5   5]
 [  2  41  24 210   3 110]]


In [9]:
res

[DetectOutput(name='test_pred', train=None, test=Metric(acc=0.6356152512998267, precision=0.633540572671102, recall=0.6356152512998267, f1=0.5818964855507842, auc=-1.0, conf_m=array([[387,   3,   0,   0,   0,   0],
        [ 11, 340,  10,  15,   2,  12],
        [  5,  20, 277,  79,   8,   1],
        [  1,   0,  28, 348,   2,  11],
        [  2,  26, 151, 169,   5,   5],
        [  2,  41,  24, 210,   3, 110]])))]

ProtNet

In [11]:
from mgtbench import AutoDetector, AutoExperiment
method = 'baseline'
model= '/data1/models/roberta-base'
nclass = len(set(data['train'][0]['label']))

metric = AutoDetector.from_detector_name(method, model_name_or_path=model,num_labels=nclass,kshot=5)
experiment = AutoExperiment.from_experiment_name('fewshot',detector=[metric])
config = {'need_finetune': False,
          'need_save': False,
          'epochs': 1,
          'lr': 5e-6,
          'batch_size':64,
          'save_path': '/data1/lyl/mgtout-1/',
          'eval':True,
          'lr_factor': 4,
          'kshot':5,
          'classifier':'MLP'
        }
# cfg = SupervisedConfig()
# cfg.update(config)
experiment.load_data(data)
res = experiment.launch(**config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /data1/models/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


length is set to 512
Calculate result for each data point
Running prediction of detector baseline
Fine-tune finished
Predict testing data
Run classification for results
[[ 73  68  10  29  70 140]
 [ 51  55  22  55  57 150]
 [ 19  66  94  86  49  76]
 [ 12  23  60 124  58 113]
 [ 20  54  55  76  71  82]
 [ 23  36  25  63  79 164]]


In [12]:
res

[DetectOutput(name='test_pred', train=None, test=Metric(acc=0.2517331022530329, precision=0.2680819878487759, recall=0.2517331022530329, f1=0.24755593478277405, auc=-1.0, conf_m=array([[ 73,  68,  10,  29,  70, 140],
        [ 51,  55,  22,  55,  57, 150],
        [ 19,  66,  94,  86,  49,  76],
        [ 12,  23,  60, 124,  58, 113],
        [ 20,  54,  55,  76,  71,  82],
        [ 23,  36,  25,  63,  79, 164]])))]